In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

### 1. Dataload

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [16]:
print(len(training_data), len(test_data))
print(type(training_data))
type(training_data[0])

60000 10000
<class 'torchvision.datasets.mnist.FashionMNIST'>


tuple

In [17]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

### 2. Model Build

In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

### 3. Optimize Parameters

In [21]:
lr = 1e-3 # 0.001
batch_size = 64 # 64개 단위로 계산하겠단거임
epochs = 3

In [22]:
#1. 손실함수 정의
loss_fn = nn.CrossEntropyLoss()
#2. optimizer 정의
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

- train_loop : 파라미터 최적화 코드
- test_loop : 테스트 데이터 성능 체크

> - train_loop는 세단계로 이루어짐
>    - optimizer.zero_grad() : 모델 매개변수 변화도 재설정. 기본적으로 변화도는 더해지기 떄문에, 중복 계산을 막기 위해 반복때마다 명시적으로 0으로 설정
>    - loss.backward() : 역전파
>    - optimizer.step() : 역전파 단계에서 수집된 변화도로 매개변수 조정 

In [24]:
def train_loop(dataloader, model, loss_fn, optimizer):
    
    size = len(dataloader.dataset)
    
    for batch_num, (x,y) in enumerate(dataloader) : # (x,y)는 배치 하나에 있는 x와 y 의미
        
        # 예측마다 손실 계산
        pred = model(x)
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_num % 100 == 0 :
            loss, current = loss.item(), batch_num * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [25]:
def test_loop(dataloader, model, loss_fn):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad(): # test data는 gradient update를 안하니까
        for X, y in dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            # 여기서는 이미지 분류라 10개 값 담겨있는 벡터 중 가장 큰 값을 고른단 의미임

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실함수와 옵티마이저를 초기화하고 train_loop와 test_loop에 전달한다. 모델의 성능 향상을 알아보기 위해 자유롭게 에폭 수를 증가시켜 볼 수 있다

In [26]:
# 반복할 epoch 수 설정 후 train loop, test loop 함수를 실행한다
epochs = 3
for i in range(epochs):
    print(f"Epoch {i+1}\n--------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('All done')

Epoch 1
--------------------
loss: 2.290699  [    0/60000]
loss: 2.280282  [ 6400/60000]
loss: 2.261385  [12800/60000]
loss: 2.263940  [19200/60000]
loss: 2.243207  [25600/60000]
loss: 2.212568  [32000/60000]
loss: 2.220546  [38400/60000]
loss: 2.176979  [44800/60000]
loss: 2.172220  [51200/60000]
loss: 2.152427  [57600/60000]
Test Error: 
 Accuracy: 49.2%, Avg loss: 2.141515 

Epoch 2
--------------------
loss: 2.149046  [    0/60000]
loss: 2.137943  [ 6400/60000]
loss: 2.079271  [12800/60000]
loss: 2.105957  [19200/60000]
loss: 2.048393  [25600/60000]
loss: 1.986339  [32000/60000]
loss: 2.015144  [38400/60000]
loss: 1.925609  [44800/60000]
loss: 1.926784  [51200/60000]
loss: 1.870091  [57600/60000]
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.863446 

Epoch 3
--------------------
loss: 1.890859  [    0/60000]
loss: 1.863777  [ 6400/60000]
loss: 1.739565  [12800/60000]
loss: 1.798853  [19200/60000]
loss: 1.686831  [25600/60000]
loss: 1.634733  [32000/60000]
loss: 1.657842  [38400/60000]